In [ ]:
import kfilter as kf
import numpy as np
import math
import matplotlib.pyplot as plt
import time

import matplotlib.animation as anm

dt = 0.05

def f(a, u):
    x = a[0]
    y = a[1]
    sita = a[2]
    v = a[3]
    dsita = a[4]
    return np.array([x + math.cos(sita)*v*dt,
                     y + math.sin(sita)*v*dt,
                     sita + dsita*dt,
                     v,
                     dsita,                   ])

def h(a):
    x = a[0]
    y = a[1]
    return np.array([x, y])

b = np.array([[1], 
              [1],
              [1],
              [1],
              [1]])
v = np.array([[10**(-3)]])
w = np.array([[1, 0], 
              [0, 1]])
x = np.array([0, 0, 0, 0, 0])
p = np.eye(5)*1000

ukf = kf.ExtendedKalmanFilter_MultipleObservation()
ukf.transition_function = f
ukf.observation_function = h
ukf.noise_model_matrix = b
ukf.system_noise_covariance_matrix = v
ukf.observation_covariance_matrix = w
ukf.prediction_state_vector = x
ukf.prediction_covariance_matrix = p

true_x = np.array([10, 10, 45/180*math.pi, 2, 10/180*math.pi])

plot_data_x = []
plot_data_y = []
plot_data_x_ = []
plot_data_y_ = []
plot_data_x_ob = []
plot_data_y_ob = []
plot_figure_x_max = 0
plot_figure_x_min = 0
plot_figure_y_max = 0
plot_figure_y_min = 0


def figre_frame(i):
    global ukf
    global true_x
    global plot_data_x, plot_data_y
    global plot_data_x_, plot_data_y_
    global plot_data_x_ob, plot_data_y_ob    
    global plot_figure_x_max, plot_figure_x_min, plot_figure_y_max, plot_figure_y_min

    true_x = f(true_x, None)
    obs = h(true_x) + np.random.normal(0, 1, 2)
    ukf.update(obs)

    plot_data_x_ob.append(obs[0])
    plot_data_y_ob.append(obs[1])
    plot_data_x.append(true_x[0])
    plot_data_y.append(true_x[1])
    plot_data_x_.append(ukf.prediction_state_vector[0])
    plot_data_y_.append(ukf.prediction_state_vector[1])

    plot_x_max = np.max(plot_data_x_ob)
    plot_x_min = np.min(plot_data_x_ob)
    plot_y_max = np.max(plot_data_y_ob)
    plot_y_min = np.min(plot_data_y_ob)

    target_x_max = plot_x_max + 0.2*(plot_x_max-plot_x_min)
    target_x_min = plot_x_min - 0.2*(plot_x_max-plot_x_min)
    target_y_max = plot_y_max + 0.2*(plot_x_max-plot_y_min)
    target_y_min = plot_y_min - 0.2*(plot_x_max-plot_y_min)

    a = 0.75

    plot_figure_x_max = plot_figure_x_max*a + (1-a)*target_x_max
    plot_figure_x_min = plot_figure_x_min*a + (1-a)*target_x_min
    plot_figure_y_max = plot_figure_y_max*a + (1-a)*target_y_max
    plot_figure_y_min = plot_figure_y_min*a + (1-a)*target_y_min

    plt.xlim(plot_figure_x_min, plot_figure_x_max)
    plt.ylim(plot_figure_y_min, plot_figure_y_max)
    plt.plot(plot_data_x, plot_data_y, c="blue")
    plt.scatter(plot_data_x_ob, plot_data_y_ob, s=2, c="gray")
    plt.plot(plot_data_x_, plot_data_y_, c="red")
    plt.title(f"{i}tick目")



fig = plt.figure()


ani = anm.FuncAnimation(fig, figre_frame, interval = dt*1000, frames = 300)

ani.save(f"./gifs/KF{time.time()}.gif", writer = 'imagemagick')

# for i in range(1000):
#     figre_frame(i)

# true_x


In [ ]:
import math
from typing import Callable

particle = np.array([[0]*50, [0]*50])
def translate_function(x, v):
    a = x[0]
    b = x[1]
    return np.array([a+1, b+1]) + v

def q():
    return np.random.normal(0, 1, 2)

def g(y, x):
    return y-x

def equal_interval_restoration_extraction(likelihood_list:list|np.ndarray) -> list:
    n = len(likelihood_list)
    normal_likelihood_list = np.asarray(likelihood_list)/sum(likelihood_list)
    distribution_list = []
    sum_likelihood = 0
    for one_likelihood in normal_likelihood_list:
        sum_likelihood += one_likelihood
        distribution_list.append(sum_likelihood)
    extraction_point_list = np.arange(n+1)[1:]/(n+1) #(0,1)端を入れず均等に分ける
    index = 0
    result_index_list = []
    for extraction_point in extraction_point_list:
        while distribution_list[index] <= extraction_point: index += 1
        result_index_list.append(index)
    return result_index_list

def update(particle:np.ndarray, 
           y:np.ndarray,
           f:Callable[[np.ndarray, np.ndarray], np.ndarray],
           g:Callable[[np.ndarray, np.ndarray], np.ndarray], 
           q:Callable[[], np.ndarray],) -> ...:
    translate_function = lambda x: f(x, q()) #システムノイズを分布に従い生成
    one_step_ahead_particle = np.asarray(list(map(translate_function, particle.T))).T
    g_ = lambda x: g(y, x)
    particle_obs = np.asarray(list(map(g_, one_step_ahead_particle.T))).T
    a = np.exp(-particle_obs**2)/math.sqrt(2*math.pi)
    b = a[0]*a[1]
    new_particle_index_list = equal_interval_restoration_extraction(b)
    new_particle = np.stack(tuple(one_step_ahead_particle.T[index] for index in new_particle_index_list)).T
    return new_particle
    

y = [-10, 10]
for i in range(10):
    y = translate_function(y, np.array([0, 0]))
    new_particle = update(particle=particle,
        f=translate_function,
        y=y,
        g=g,
        q=q)
    particle = new_particle
print(particle)

In [57]:
# import numpy.testing as nptype
def equal_interval_restoration_extraction(likelihood_list:list|np.ndarray) -> list:
    n = len(likelihood_list)
    normal_likelihood_list = np.asarray(likelihood_list)/sum(likelihood_list)
    distribution_list = []
    sum_likelihood = 0
    for one_likelihood in normal_likelihood_list:
        sum_likelihood += one_likelihood
        distribution_list.append(sum_likelihood)
    extraction_point_list = np.arange(n+1)[1:]/(n+1) #(0,1)端を入れず均等に分ける
    index = 0
    result_index_list = []
    for extraction_point in extraction_point_list:
        while distribution_list[index] <= extraction_point: index += 1
        result_index_list.append(index)
    return result_index_list

likelihood_list = np.random.random(3)
print(likelihood_list)
equal_interval_restoration_extraction(likelihood_list)

[0.17052118 0.36831574 0.92882341]


[1, 2, 2]

In [7]:
import sympy as sp
from typing import Literal

def moment(f:Literal["formula of Sympy"], n:int) -> float:
    """モーメントを求める。
    fはxの確率密度関数とする。
    nはモーメントの次数を表す。"""
    t, x = sp.symbols("t,x")
    expectation = sp.integrate(sp.exp(t*x)*f, 
                               (x, -sp.oo, sp.oo)) #母関数E[e^{tx}]を求める
    taylor_expectation = sp.series(expectation, t, 0, n+1) #n次までマクローリン展開
    coefficient = taylor_expectation.coeff(t, n) #n次係数の取り出し
    n_moment = coefficient*sp.factorial(n) #n!をかけてモーメントに。
    return float(n_moment)

x = sp.Symbol("x")
f = sp.exp(-x**2/2)/sp.sqrt(2*sp.pi)

moment(f, 5)

0.0